# Part 4: Entity Resolution — 중복 엔티티 통합

**소요시간:** 1시간  
**난이도:** ★★★☆  
**마일스톤:** 정제된 KG — 중복 제거 완료 (예: 45개 → 30개 노드)

---

## 학습 목표

1. **중복 엔티티**가 Knowledge Graph 품질에 미치는 영향 이해
2. **문자열 유사도** 기반 Entity Resolution (rapidfuzz)
3. **임베딩 기반** Entity Resolution (OpenAI Embeddings + 코사인 유사도)
4. Neo4j에서 **노드 통합** (MERGE + 관계 재연결)
5. 통합 전/후 **쿼리 품질 비교**

---
## 1. 환경 설정

Neo4j에 연결하고, Part 3에서 적재한 기존 그래프를 확인합니다.

In [ ]:
# 필수 패키지 임포트
import json
import os
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from openai import OpenAI
from neo4j import GraphDatabase
from rapidfuzz import fuzz, process
from dotenv import load_dotenv

# 한글 폰트 설정
matplotlib.rcParams['font.family'] = 'AppleGothic'  # macOS
# matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # Windows
matplotlib.rcParams['axes.unicode_minus'] = False

print("패키지 로드 완료")

In [ ]:
# 환경변수 및 연결
load_dotenv()

# OpenAI (임베딩용)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Neo4j 연결
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "graphrag2024")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 연결 테스트
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    print(f"Neo4j 연결 성공: {result.single()['test']}")

In [ ]:
# 기존 그래프 상태 확인
with driver.session() as session:
    # 전체 노드/관계 수
    node_count = session.run("MATCH (n) RETURN count(n) AS cnt").single()["cnt"]
    rel_count = session.run("MATCH ()-[r]->() RETURN count(r) AS cnt").single()["cnt"]
    
    # 모든 엔티티 이름 조회
    all_entities = session.run("""
        MATCH (n) 
        RETURN n.name AS name, labels(n) AS labels 
        ORDER BY n.name
    """).data()

print(f"현재 그래프 상태:")
print(f"  노드: {node_count}개")
print(f"  관계: {rel_count}개")
print(f"\n전체 엔티티 목록 ({len(all_entities)}개):")
for e in all_entities:
    print(f"  - {e['name']} [{', '.join(e['labels'])}]")

---
## 2. 중복 문제 이해

LLM이 추출한 엔티티에는 **같은 대상을 다르게 표현한 중복**이 빈번합니다.

예시:
- "삼성전자", "Samsung Electronics", "Samsung", "삼성"
- "SK하이닉스", "SK Hynix", "하이닉스"
- "네이버", "NAVER", "Naver"

이런 중복은 **Multi-hop 쿼리의 정확도를 떨어뜨립니다.**

In [ ]:
# 중복 문제 시연을 위해 의도적으로 중복 엔티티를 추가
# (Part 3에서 이미 LLM 추출 결과가 있다면 이 셀은 건너뛰어도 됩니다)

duplicate_entities = [
    # 삼성전자 변형
    ("Samsung Electronics", "Company"),
    ("Samsung", "Company"),
    ("삼성", "Company"),
    # SK하이닉스 변형
    ("SK Hynix", "Company"),
    ("하이닉스", "Company"),
    # 네이버 변형
    ("NAVER", "Company"),
    ("Naver", "Company"),
    # 엔비디아 변형
    ("NVIDIA", "Company"),
    ("Nvidia", "Company"),
    # 하이퍼클로바 변형
    ("HyperCLOVA X", "Product"),
    ("HyperClova X", "Product"),
]

with driver.session() as session:
    for name, label in duplicate_entities:
        session.run(
            f"MERGE (n:{label} {{name: $name}})",
            name=name
        )
    
    # 일부 중복 엔티티에 관계 추가 (문제를 더 현실적으로)
    session.run("""
        MATCH (a:Company {name: "Samsung Electronics"})
        MATCH (b:Company {name: "NVIDIA"})
        MERGE (a)-[:PARTNERS_WITH]->(b)
    """)
    session.run("""
        MATCH (a:Company {name: "NAVER"})
        MATCH (b:Product {name: "HyperCLOVA X"})
        MERGE (a)-[:DEVELOPS]->(b)
    """)
    
    # 변경 후 노드 수 확인
    new_count = session.run("MATCH (n) RETURN count(n) AS cnt").single()["cnt"]

print(f"중복 엔티티 추가 완료")
print(f"  이전 노드 수: {node_count}")
print(f"  현재 노드 수: {new_count}")
print(f"  추가된 중복: {new_count - node_count}개")

In [ ]:
# 중복이 Multi-hop 쿼리에 미치는 영향 시연
with driver.session() as session:
    # 쿼리 1: "삼성전자와 관련된 모든 엔티티"
    result1 = session.run("""
        MATCH (samsung:Company {name: "삼성전자"})-[r]-(connected)
        RETURN samsung.name AS source, type(r) AS relation, connected.name AS target
    """).data()
    
    # 쿼리 2: "Samsung Electronics와 관련된 모든 엔티티" (다른 결과!)
    result2 = session.run("""
        MATCH (samsung:Company {name: "Samsung Electronics"})-[r]-(connected)
        RETURN samsung.name AS source, type(r) AS relation, connected.name AS target
    """).data()
    
    # 쿼리 3: 2-hop 연결 (삼성전자 → ? → ?)
    result3 = session.run("""
        MATCH path = (samsung:Company {name: "삼성전자"})-[*1..2]-(target)
        RETURN DISTINCT target.name AS reachable
    """).data()

print("중복이 쿼리에 미치는 영향:")
print(f"\n1) '삼성전자'로 검색한 관계: {len(result1)}개")
for r in result1:
    print(f"   {r['source']} --[{r['relation']}]--> {r['target']}")

print(f"\n2) 'Samsung Electronics'로 검색한 관계: {len(result2)}개")
for r in result2:
    print(f"   {r['source']} --[{r['relation']}]--> {r['target']}")

print(f"\n3) '삼성전자'에서 2-hop 도달 가능 엔티티: {len(result3)}개")
for r in result3:
    print(f"   - {r['reachable']}")

print(f"\n문제: 같은 기업인데 다른 이름으로 검색하면 다른 결과가 나옵니다!")
print(f"Samsung Electronics의 관계가 삼성전자 검색에 포함되지 않습니다.")

---
## 3. 문자열 유사도 기반 Entity Resolution

**rapidfuzz** 라이브러리를 사용하여 이름이 비슷한 엔티티 쌍을 찾습니다.

| 알고리즘 | 특징 | 적합한 경우 |
|----------|------|------------|
| Levenshtein | 편집 거리 기반 | 오타, 띄어쓰기 차이 |
| Jaro-Winkler | 접두사 가중치 | 약어, 접두사 공유 |
| Token Sort Ratio | 토큰 정렬 후 비교 | 단어 순서 다른 경우 |

In [ ]:
# Neo4j에서 모든 엔티티 이름 가져오기
with driver.session() as session:
    entities_data = session.run("""
        MATCH (n)
        RETURN n.name AS name, labels(n) AS labels, id(n) AS node_id
        ORDER BY n.name
    """).data()

entity_names = [e["name"] for e in entities_data if e["name"]]
print(f"총 엔티티: {len(entity_names)}개")
print(f"\n엔티티 목록:")
for name in sorted(entity_names):
    print(f"  - {name}")

In [ ]:
# 문자열 유사도 함수들 비교
test_pairs = [
    ("삼성전자", "Samsung Electronics"),
    ("삼성전자", "삼성"),
    ("삼성전자", "Samsung"),
    ("SK하이닉스", "SK Hynix"),
    ("SK하이닉스", "하이닉스"),
    ("네이버", "NAVER"),
    ("네이버", "Naver"),
    ("하이퍼클로바X", "HyperCLOVA X"),
    ("엔비디아", "NVIDIA"),
    ("삼성전자", "LG전자"),  # 이건 다른 엔티티!
]

print(f"{'쌍':<35} {'Levenshtein':>12} {'Jaro-Winkler':>13} {'Token Sort':>11}")
print("-" * 75)

for a, b in test_pairs:
    lev = fuzz.ratio(a, b)
    jaro = fuzz.WRatio(a, b)  # Weighted Ratio (다양한 방법 중 최선)
    token_sort = fuzz.token_sort_ratio(a, b)
    
    pair_str = f"{a} vs {b}"
    print(f"{pair_str:<35} {lev:>11.1f} {jaro:>12.1f} {token_sort:>10.1f}")

In [ ]:
# 임계값 실험: 0.7, 0.8, 0.9
def find_similar_pairs(names: list[str], threshold: float = 0.8) -> list[tuple]:
    """엔티티 이름 리스트에서 유사한 쌍을 찾습니다.
    
    Args:
        names: 엔티티 이름 리스트
        threshold: 유사도 임계값 (0~100)
    
    Returns:
        (이름1, 이름2, 유사도) 튜플 리스트
    """
    pairs = []
    for i, name_a in enumerate(names):
        for j, name_b in enumerate(names):
            if i >= j:  # 중복 방지
                continue
            
            # 여러 유사도 메트릭 중 최대값 사용
            score = max(
                fuzz.ratio(name_a, name_b),
                fuzz.WRatio(name_a, name_b),
                fuzz.token_sort_ratio(name_a, name_b)
            )
            
            if score >= threshold:
                pairs.append((name_a, name_b, score))
    
    return sorted(pairs, key=lambda x: -x[2])

# 임계값별 결과 비교
for threshold in [70, 80, 90]:
    pairs = find_similar_pairs(entity_names, threshold)
    print(f"\n임계값 {threshold}% — 유사 쌍: {len(pairs)}개")
    for a, b, score in pairs:
        print(f"  [{score:.0f}%] {a} ≈ {b}")

In [ ]:
# 후보 쌍 생성 함수 (프로덕션 버전)
def generate_candidate_pairs(
    names: list[str], 
    threshold: float = 80,
    method: str = "combined"
) -> list[dict]:
    """Entity Resolution 후보 쌍을 생성합니다.
    
    Args:
        names: 엔티티 이름 리스트
        threshold: 유사도 임계값
        method: 유사도 계산 방법 ("levenshtein", "jaro", "token", "combined")
    
    Returns:
        후보 쌍 딕셔너리 리스트
    """
    candidates = []
    
    for i, name_a in enumerate(names):
        for j, name_b in enumerate(names):
            if i >= j:
                continue
            
            scores = {
                "levenshtein": fuzz.ratio(name_a, name_b),
                "jaro": fuzz.WRatio(name_a, name_b),
                "token_sort": fuzz.token_sort_ratio(name_a, name_b),
                "partial": fuzz.partial_ratio(name_a, name_b)
            }
            
            if method == "combined":
                final_score = max(scores.values())
            else:
                final_score = scores.get(method, 0)
            
            if final_score >= threshold:
                candidates.append({
                    "entity_a": name_a,
                    "entity_b": name_b,
                    "score": final_score,
                    "scores": scores
                })
    
    return sorted(candidates, key=lambda x: -x["score"])

# 임계값 80으로 후보 쌍 생성
string_candidates = generate_candidate_pairs(entity_names, threshold=80)

print(f"문자열 유사도 기반 후보 쌍: {len(string_candidates)}개")
df_string = pd.DataFrame([
    {"엔티티 A": c["entity_a"], "엔티티 B": c["entity_b"], "최종 점수": c["score"],
     **c["scores"]}
    for c in string_candidates
])
df_string

### 관찰

문자열 유사도의 **한계**:
- "삼성전자" vs "Samsung Electronics" → 문자열이 전혀 달라서 낮은 점수
- 한글-영문 변환을 감지하지 못함
- 약어(Samsung → 삼성)를 이해하지 못함

**해결:** 임베딩 기반 유사도로 **의미적 유사성**을 포착합니다.

---
## 4. 임베딩 기반 Entity Resolution

OpenAI Embedding 모델을 사용하여 엔티티 이름을 **벡터로 변환**하고,  
**코사인 유사도**로 의미적으로 같은 엔티티를 찾습니다.

In [ ]:
def get_embeddings(texts: list[str], model: str = "text-embedding-3-small") -> np.ndarray:
    """텍스트 리스트의 임베딩을 생성합니다.
    
    Args:
        texts: 텍스트 리스트
        model: 임베딩 모델명
    
    Returns:
        임베딩 행렬 (N x D)
    """
    response = client.embeddings.create(
        input=texts,
        model=model
    )
    
    embeddings = [item.embedding for item in response.data]
    return np.array(embeddings)

# 모든 엔티티 이름 임베딩
print(f"엔티티 {len(entity_names)}개 임베딩 생성 중...")
embeddings = get_embeddings(entity_names)
print(f"임베딩 행렬 크기: {embeddings.shape}")
print(f"임베딩 차원: {embeddings.shape[1]}")

In [ ]:
def cosine_similarity_matrix(embeddings: np.ndarray) -> np.ndarray:
    """임베딩 행렬의 코사인 유사도 행렬을 계산합니다.
    
    Args:
        embeddings: (N, D) 임베딩 행렬
    
    Returns:
        (N, N) 코사인 유사도 행렬
    """
    # L2 정규화
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized = embeddings / norms
    
    # 코사인 유사도 = 내적 (정규화 후)
    similarity = normalized @ normalized.T
    return similarity

# 유사도 행렬 계산
sim_matrix = cosine_similarity_matrix(embeddings)
print(f"유사도 행렬 크기: {sim_matrix.shape}")

# 히트맵 시각화
fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(sim_matrix, cmap="YlOrRd", vmin=0.5, vmax=1.0)
ax.set_xticks(range(len(entity_names)))
ax.set_yticks(range(len(entity_names)))
ax.set_xticklabels(entity_names, rotation=45, ha="right", fontsize=8)
ax.set_yticklabels(entity_names, fontsize=8)
ax.set_title("엔티티 임베딩 코사인 유사도 히트맵")
plt.colorbar(im, ax=ax, label="코사인 유사도")
plt.tight_layout()
plt.show()

In [ ]:
def find_embedding_pairs(
    names: list[str], 
    sim_matrix: np.ndarray, 
    threshold: float = 0.85
) -> list[dict]:
    """임베딩 유사도 기반으로 후보 쌍을 찾습니다.
    
    Args:
        names: 엔티티 이름 리스트
        sim_matrix: 코사인 유사도 행렬
        threshold: 유사도 임계값 (0~1)
    
    Returns:
        후보 쌍 딕셔너리 리스트
    """
    pairs = []
    n = len(names)
    
    for i in range(n):
        for j in range(i + 1, n):
            score = sim_matrix[i, j]
            if score >= threshold:
                pairs.append({
                    "entity_a": names[i],
                    "entity_b": names[j],
                    "cosine_similarity": round(float(score), 4)
                })
    
    return sorted(pairs, key=lambda x: -x["cosine_similarity"])

# 임베딩 기반 후보 쌍 (임계값 0.85)
embedding_candidates = find_embedding_pairs(entity_names, sim_matrix, threshold=0.85)

print(f"임베딩 기반 후보 쌍: {len(embedding_candidates)}개")
for c in embedding_candidates:
    print(f"  [{c['cosine_similarity']:.4f}] {c['entity_a']} ≈ {c['entity_b']}")

In [ ]:
# 클러스터링으로 동일 엔티티 그룹화
def cluster_entities(
    names: list[str], 
    sim_matrix: np.ndarray, 
    threshold: float = 0.85
) -> list[list[str]]:
    """유사한 엔티티를 클러스터로 그룹화합니다.
    
    단순 연결 컴포넌트 방식: A≈B, B≈C이면 {A,B,C}는 같은 그룹
    
    Args:
        names: 엔티티 이름 리스트
        sim_matrix: 코사인 유사도 행렬
        threshold: 유사도 임계값
    
    Returns:
        엔티티 이름 클러스터 리스트 (2개 이상인 그룹만)
    """
    n = len(names)
    # Union-Find
    parent = list(range(n))
    
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    
    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py
    
    # 임계값 이상인 쌍을 연결
    for i in range(n):
        for j in range(i + 1, n):
            if sim_matrix[i, j] >= threshold:
                union(i, j)
    
    # 그룹 생성
    groups = defaultdict(list)
    for i in range(n):
        groups[find(i)].append(names[i])
    
    # 2개 이상인 그룹만 반환
    return [group for group in groups.values() if len(group) >= 2]

# 클러스터링 실행
clusters = cluster_entities(entity_names, sim_matrix, threshold=0.85)

print(f"동일 엔티티 클러스터: {len(clusters)}개 그룹")
for i, cluster in enumerate(clusters, 1):
    canonical = cluster[0]  # 첫 번째를 대표 이름으로
    print(f"\n  그룹 {i} (대표: {canonical}):")
    for name in cluster:
        marker = " [대표]" if name == canonical else ""
        print(f"    - {name}{marker}")

In [ ]:
# 문자열 vs 임베딩 방식 비교
print("방식 비교: 문자열 유사도 vs 임베딩 유사도")
print("=" * 60)

# 문자열 기반 결과
string_pairs_set = {
    (c["entity_a"], c["entity_b"]) for c in string_candidates
}

# 임베딩 기반 결과
embedding_pairs_set = {
    (c["entity_a"], c["entity_b"]) for c in embedding_candidates
}

# 비교
only_string = string_pairs_set - embedding_pairs_set
only_embedding = embedding_pairs_set - string_pairs_set
both = string_pairs_set & embedding_pairs_set

print(f"\n문자열만 감지: {len(only_string)}개")
for a, b in only_string:
    print(f"  {a} ≈ {b}")

print(f"\n임베딩만 감지: {len(only_embedding)}개")
for a, b in only_embedding:
    print(f"  {a} ≈ {b}")

print(f"\n양쪽 모두 감지: {len(both)}개")
for a, b in both:
    print(f"  {a} ≈ {b}")

print(f"\n결론:")
print(f"- 임베딩은 한글-영문 변환을 잘 감지 (의미적 유사도)")
print(f"- 문자열은 오타, 띄어쓰기 차이를 잘 감지 (형태적 유사도)")
print(f"- 두 방식을 결합하면 최고의 결과를 얻을 수 있습니다")

---
## 5. Neo4j에서 노드 통합

클러스터링 결과를 바탕으로 중복 노드를 **하나로 통합**합니다.

통합 과정:
1. 대표 노드 선정 (가장 한국어스러운 이름 / 가장 많은 관계를 가진 노드)
2. 중복 노드의 속성을 대표 노드로 복사
3. 중복 노드의 관계를 대표 노드로 재연결
4. 중복 노드 삭제

In [ ]:
def select_canonical_name(names: list[str]) -> str:
    """클러스터에서 대표 이름을 선정합니다.
    
    우선순위:
    1. 한글 이름 (한국어 KG이므로)
    2. 더 긴 이름 (더 공식적인 경향)
    
    Args:
        names: 같은 엔티티의 이름 리스트
    
    Returns:
        대표 이름
    """
    def score(name):
        # 한글 포함 여부 (높을수록 좋음)
        has_korean = any('가' <= c <= '힣' for c in name)
        korean_score = 100 if has_korean else 0
        # 길이 (길수록 공식 명칭에 가까움)
        length_score = len(name)
        return korean_score + length_score
    
    return max(names, key=score)

# 대표 이름 선정 테스트
for cluster in clusters:
    canonical = select_canonical_name(cluster)
    others = [n for n in cluster if n != canonical]
    print(f"  대표: '{canonical}' ← 통합: {others}")

In [ ]:
def merge_entities_in_neo4j(
    driver, 
    canonical_name: str, 
    duplicate_names: list[str]
) -> dict:
    """Neo4j에서 중복 엔티티를 대표 엔티티로 통합합니다.
    
    Args:
        driver: Neo4j 드라이버
        canonical_name: 대표 엔티티 이름
        duplicate_names: 통합될 중복 엔티티 이름 리스트
    
    Returns:
        통합 결과 딕셔너리
    """
    stats = {"relations_moved": 0, "nodes_deleted": 0, "properties_merged": 0}
    
    with driver.session() as session:
        for dup_name in duplicate_names:
            if dup_name == canonical_name:
                continue
            
            # 1. 중복 노드의 속성을 대표 노드로 복사
            session.run("""
                MATCH (canonical {name: $canonical_name})
                MATCH (dup {name: $dup_name})
                SET canonical += properties(dup)
                SET canonical.name = $canonical_name
                SET canonical.aliases = coalesce(canonical.aliases, []) + [$dup_name]
            """, canonical_name=canonical_name, dup_name=dup_name)
            stats["properties_merged"] += 1
            
            # 2. 나가는 관계 재연결 (dup → X 를 canonical → X 로)
            result = session.run("""
                MATCH (dup {name: $dup_name})-[r]->(target)
                MATCH (canonical {name: $canonical_name})
                WITH canonical, target, type(r) AS relType, properties(r) AS props
                CALL apoc.create.relationship(canonical, relType, props, target) YIELD rel
                RETURN count(rel) AS cnt
            """, dup_name=dup_name, canonical_name=canonical_name)
            cnt = result.single()["cnt"]
            stats["relations_moved"] += cnt
            
            # 3. 들어오는 관계 재연결 (X → dup 를 X → canonical 로)
            result = session.run("""
                MATCH (source)-[r]->(dup {name: $dup_name})
                MATCH (canonical {name: $canonical_name})
                WITH source, canonical, type(r) AS relType, properties(r) AS props
                CALL apoc.create.relationship(source, relType, props, canonical) YIELD rel
                RETURN count(rel) AS cnt
            """, dup_name=dup_name, canonical_name=canonical_name)
            cnt = result.single()["cnt"]
            stats["relations_moved"] += cnt
            
            # 4. 중복 노드 삭제
            session.run("""
                MATCH (dup {name: $dup_name})
                DETACH DELETE dup
            """, dup_name=dup_name)
            stats["nodes_deleted"] += 1
    
    return stats

print("통합 함수 정의 완료")
print("주의: 이 함수는 APOC 플러그인이 필요합니다.")
print("APOC 없이 실행하려면 아래의 대체 함수를 사용하세요.")

In [ ]:
# APOC 없이 사용 가능한 대체 함수
def merge_entities_basic(
    driver, 
    canonical_name: str, 
    duplicate_names: list[str]
) -> dict:
    """APOC 없이 순수 Cypher로 엔티티를 통합합니다.
    
    제약: 관계 타입을 동적으로 생성할 수 없으므로,
    알려진 관계 타입을 명시적으로 처리합니다.
    """
    KNOWN_REL_TYPES = [
        "DEVELOPS", "INVESTS_IN", "COMPETES_WITH", "PARTNERS_WITH",
        "LEADS", "LOCATED_AT", "SUPPLIES_TO", "USES"
    ]
    
    stats = {"relations_moved": 0, "nodes_deleted": 0}
    
    with driver.session() as session:
        for dup_name in duplicate_names:
            if dup_name == canonical_name:
                continue
            
            # 속성 복사 + aliases 추가
            session.run("""
                MATCH (canonical {name: $canonical_name})
                MATCH (dup {name: $dup_name})
                SET canonical += properties(dup)
                SET canonical.name = $canonical_name
                SET canonical.aliases = coalesce(canonical.aliases, []) + [$dup_name]
            """, canonical_name=canonical_name, dup_name=dup_name)
            
            # 각 관계 타입별로 재연결
            for rel_type in KNOWN_REL_TYPES:
                # 나가는 관계
                result = session.run(f"""
                    MATCH (dup {{name: $dup_name}})-[r:{rel_type}]->(target)
                    MATCH (canonical {{name: $canonical_name}})
                    MERGE (canonical)-[:{rel_type}]->(target)
                    DELETE r
                    RETURN count(r) AS cnt
                """, dup_name=dup_name, canonical_name=canonical_name)
                stats["relations_moved"] += result.single()["cnt"]
                
                # 들어오는 관계
                result = session.run(f"""
                    MATCH (source)-[r:{rel_type}]->(dup {{name: $dup_name}})
                    MATCH (canonical {{name: $canonical_name}})
                    MERGE (source)-[:{rel_type}]->(canonical)
                    DELETE r
                    RETURN count(r) AS cnt
                """, dup_name=dup_name, canonical_name=canonical_name)
                stats["relations_moved"] += result.single()["cnt"]
            
            # 중복 노드 삭제
            session.run("""
                MATCH (dup {name: $dup_name})
                DETACH DELETE dup
            """, dup_name=dup_name)
            stats["nodes_deleted"] += 1
    
    return stats

print("기본 통합 함수(APOC 불필요) 정의 완료")

In [ ]:
# 통합 전 상태 기록
with driver.session() as session:
    before_nodes = session.run("MATCH (n) RETURN count(n) AS cnt").single()["cnt"]
    before_rels = session.run("MATCH ()-[r]->() RETURN count(r) AS cnt").single()["cnt"]

print(f"통합 전 상태:")
print(f"  노드: {before_nodes}개")
print(f"  관계: {before_rels}개")

# 클러스터별 통합 실행
total_stats = {"relations_moved": 0, "nodes_deleted": 0}

for cluster in clusters:
    canonical = select_canonical_name(cluster)
    duplicates = [n for n in cluster if n != canonical]
    
    if not duplicates:
        continue
    
    print(f"\n통합: '{canonical}' ← {duplicates}")
    stats = merge_entities_basic(driver, canonical, duplicates)
    
    total_stats["relations_moved"] += stats["relations_moved"]
    total_stats["nodes_deleted"] += stats["nodes_deleted"]
    
    print(f"  삭제된 노드: {stats['nodes_deleted']}개")
    print(f"  이동된 관계: {stats['relations_moved']}개")

# 통합 후 상태
with driver.session() as session:
    after_nodes = session.run("MATCH (n) RETURN count(n) AS cnt").single()["cnt"]
    after_rels = session.run("MATCH ()-[r]->() RETURN count(r) AS cnt").single()["cnt"]

print(f"\n{'='*60}")
print(f"통합 결과 요약:")
print(f"  노드: {before_nodes}개 → {after_nodes}개 (△{after_nodes - before_nodes})")
print(f"  관계: {before_rels}개 → {after_rels}개 (△{after_rels - before_rels})")
print(f"  삭제된 중복 노드: {total_stats['nodes_deleted']}개")
print(f"  이동된 관계: {total_stats['relations_moved']}개")

In [ ]:
# 통합 후 Multi-hop 쿼리 재실행 → 결과 개선 확인
with driver.session() as session:
    # 이제 "삼성전자"로 검색하면 Samsung Electronics의 관계도 포함
    result_after = session.run("""
        MATCH (samsung:Company {name: "삼성전자"})-[r]-(connected)
        RETURN samsung.name AS source, type(r) AS relation, connected.name AS target
    """).data()
    
    # aliases 확인
    aliases = session.run("""
        MATCH (n)
        WHERE n.aliases IS NOT NULL AND size(n.aliases) > 0
        RETURN n.name AS name, n.aliases AS aliases
    """).data()
    
    # 2-hop 연결 재실행
    result_2hop = session.run("""
        MATCH path = (samsung:Company {name: "삼성전자"})-[*1..2]-(target)
        RETURN DISTINCT target.name AS reachable
    """).data()

print("통합 후 쿼리 결과:")
print(f"\n1) '삼성전자' 관계 (통합 후): {len(result_after)}개")
for r in result_after:
    print(f"   {r['source']} --[{r['relation']}]--> {r['target']}")

print(f"\n2) aliases가 있는 엔티티:")
for a in aliases:
    print(f"   {a['name']}: {a['aliases']}")

print(f"\n3) '삼성전자'에서 2-hop 도달 가능 (통합 후): {len(result_2hop)}개")
for r in result_2hop:
    print(f"   - {r['reachable']}")

print(f"\n이전에 'Samsung Electronics'로만 접근 가능했던 관계가")
print(f"이제 '삼성전자'로도 접근 가능합니다!")

In [ ]:
# 통합 전/후 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 노드 수 비교
categories = ["통합 전", "통합 후"]
node_values = [before_nodes, after_nodes]
rel_values = [before_rels, after_rels]

ax1 = axes[0]
bars = ax1.bar(categories, node_values, color=["#ef4444", "#22c55e"], width=0.5)
ax1.set_ylabel("노드 수")
ax1.set_title("노드 수 변화")
for bar, val in zip(bars, node_values):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
             str(val), ha='center', va='bottom', fontweight='bold')

# 관계 수 비교
ax2 = axes[1]
bars = ax2.bar(categories, rel_values, color=["#ef4444", "#22c55e"], width=0.5)
ax2.set_ylabel("관계 수")
ax2.set_title("관계 수 변화")
for bar, val in zip(bars, rel_values):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
             str(val), ha='center', va='bottom', fontweight='bold')

plt.suptitle("Entity Resolution 전/후 비교", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

reduction = ((before_nodes - after_nodes) / before_nodes * 100) if before_nodes > 0 else 0
print(f"\n노드 감소율: {reduction:.1f}%")
print(f"중복 제거로 그래프가 더 정확하고 효율적으로 변했습니다.")

---
## 6. 연습 문제

### 연습 6.1: 제조 도메인에서 중복 장비명 통합

제조 도메인 데이터에는 같은 장비를 다르게 부르는 경우가 흔합니다:  
- "MX-100" vs "원료혼합기" vs "MX100"
- "PR-200" vs "프레스기" vs "PR200"

아래 코드를 완성하여 제조 도메인의 중복 엔티티를 통합해보세요.

In [ ]:
# 연습 6.1: 제조 도메인 중복 통합

# 제조 도메인 엔티티 (실제로는 Part 3 연습에서 추출한 결과 사용)
manufacturing_entities = [
    "MX-100", "원료혼합기", "MX100",
    "PR-200", "프레스기", "PR200",
    "HT-300", "열처리로", "HT300",
    "GR-400", "연마기", "GR400",
    "QC-500", "검사장비", "QC500",
    "삼성정밀", "Samsung Precision",
    "현대위아", "Hyundai Wia",
    "포스코DX", "POSCO DX", "포스코디엑스"
]

# TODO: 임베딩 기반으로 중복 감지
print("제조 도메인 엔티티 임베딩 생성 중...")
mfg_embeddings = get_embeddings(manufacturing_entities)
mfg_sim_matrix = cosine_similarity_matrix(mfg_embeddings)

# TODO: 클러스터링
mfg_clusters = cluster_entities(manufacturing_entities, mfg_sim_matrix, threshold=0.80)

print(f"\n제조 도메인 동일 엔티티 클러스터: {len(mfg_clusters)}개")
for i, cluster in enumerate(mfg_clusters, 1):
    canonical = select_canonical_name(cluster)
    print(f"  그룹 {i}: 대표 '{canonical}' ← {[n for n in cluster if n != canonical]}")

# TODO: 임계값을 조정하여 최적 결과를 찾아보세요 (0.75, 0.80, 0.85, 0.90)

### 연습 6.2: 최적 임계값 찾기

임계값이 너무 낮으면 다른 엔티티를 통합하고 (False Positive),  
너무 높으면 같은 엔티티를 놓칩니다 (False Negative).

임계값별 클러스터 수를 비교하여 최적값을 찾아보세요.

In [ ]:
# 연습 6.2: 임계값 최적화
thresholds = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
threshold_results = []

for t in thresholds:
    clusters_t = cluster_entities(entity_names, sim_matrix, threshold=t)
    n_clusters = len(clusters_t)
    n_merged = sum(len(c) - 1 for c in clusters_t)  # 통합될 노드 수
    
    threshold_results.append({
        "임계값": t,
        "클러스터 수": n_clusters,
        "통합될 노드": n_merged,
        "잔여 노드": len(entity_names) - n_merged
    })

df_thresholds = pd.DataFrame(threshold_results)
print("임계값별 결과:")
print(df_thresholds.to_string(index=False))

# 시각화
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(thresholds, [r["클러스터 수"] for r in threshold_results], 
        marker='o', label="클러스터 수", color="#3b82f6")
ax.plot(thresholds, [r["통합될 노드"] for r in threshold_results], 
        marker='s', label="통합될 노드 수", color="#ef4444")

ax.set_xlabel("코사인 유사도 임계값")
ax.set_ylabel("개수")
ax.set_title("임계값에 따른 Entity Resolution 결과")
ax.legend()
ax.grid(True, alpha=0.3)

# 최적 임계값 표시 (팔꿈치 지점)
ax.axvline(x=0.85, color='gray', linestyle='--', alpha=0.5, label="추천 임계값")
ax.annotate('추천: 0.85', xy=(0.85, max(r["클러스터 수"] for r in threshold_results)),
            fontsize=10, ha='center')

plt.tight_layout()
plt.show()

print("\n일반적으로 0.85가 좋은 출발점입니다.")
print("도메인과 데이터에 따라 조정이 필요합니다.")

---
## 정리

### 이번 파트에서 배운 것

1. **중복 엔티티는 KG 품질을 심각하게 저하**시킨다 (특히 Multi-hop 쿼리)
2. **문자열 유사도**는 오타/띄어쓰기에 강하지만, 한글-영문 변환에 약하다
3. **임베딩 기반 유사도**는 의미적 동일성을 포착한다 (한글-영문 OK)
4. **두 방식을 결합**하면 최고의 ER 성능을 얻을 수 있다
5. **임계값 튜닝**이 중요하다 (너무 낮으면 False Positive, 너무 높으면 False Negative)
6. **aliases 필드**로 통합 이력을 보존하는 것이 좋다

### 다음 파트 예고

**Part 5: 멀티모달 VLM** - 표와 이미지가 포함된 실무 문서를 그래프로 변환합니다.

In [ ]:
# 리소스 정리
driver.close()
print("Neo4j 연결 종료")
print("Part 4 실습 완료!")